In [4]:
#                             DIRECT CONTACT MEMBRANE DISTILLATION SIMULATION PROCESS                  


#LIBRARIES IMPORT
import matplotlib.pyplot as plt 
import numpy as np
from scipy.optimize import fsolve
from datetime import datetime

# DCMD function 
def DCMD(FH0,FC0,c_salt,TH0,TC0,L,W,H,N):

    #MODEL INPUTS
    memb=1
    
    # Membrane properties
    
    if memb==1:
# PVDF
        km=3.1e-5           # Membrane Conductivity [kJ/s m K]
        em=0.000140         # Membrane Thickness [m]
        dp = 4.5e-7         # Pore size [m]     
        Por = 0.75          # Porosity (between 0-1)

    elif memb==2:
# PTFE
        km=3.1e-5           # Membrane Conductivity [kJ/s m K]
        em=0.000175         # Membrane Thickness [m]
        dp = 5e-7         # Pore size [m]     
        Por = 0.85          # Porosity (between 0-1)
    
    elif memb==3:
# Experimental membrane properties
        km=3.1e-5           # Membrane Conductivity [kJ/s m K]
        em=0.000100         # Membrane Thickness [m]
        dp = 2.2e-7         # Pore size [m]     
        Por = 0.75          # Porosity (between 0-1)

    dx=L/N              # width of the step
    n=N-1               # matrix rearrage
    Darea=dx*W          # Area Differential


# Constant
    DH=2256           # Enthalpy of vaporization         [kJ/kg]  

# Fluid Properties
    # Water density [kg/m3]
    def den(T):
            a = 999.79684
            b = 0.068317355
            c = -0.010740248
            D = 0.00082140905
            E = -0.000023030988        
            return (a + b * T + c * T**2 + D * T**2.5 + E * T**3)

# Water dynamic viscosity [Pa s]
    def d_vis(T):
            a = 0.00002414
            b = 247.8
            c = 140       
            return  a * 10**(b / (T - c + 273.15))

# Water kinematic viscosity [m2/s]
    def k_vis(T):

            return d_vis(T)/den(T)

# Water heat conductivity []
    def k_f(T):
            T = T + 273
            return (-0.000008354 * T**2 + 0.00653 * T - 0.5981) / 1000  

# Water Vapour Pressure [Pa]
    def antoine(T):
            A=6.20963
            B=2354.731
            C=7.559
            return 10**(A-B/(T+273+C))*1e5

# Water Activity f(mass fraction)
    def wa(c_salt):
            a = -0.000171349214215732
            b = -0.00468326495297634
            c = 1
            return  a * c_salt**2 + b * c_salt + c

# Water Specific Heat
    def cpw(T):
            T = (T + 273.15) / 1000
            a = -203.606
            b = 1523.29
            c = -3196.413
            D = 2474.455
            E = 3.855326


            return (a + b * T + c * T**2 + D * T**3 + E / T**2) / 18
        
# NaCl Specific Heat
    def cps(T):
        T = (T + 273.15) / 1000
        a = 50.72389
        b = 6.672267
        c = -2.517167
        D = 10.15934
        E = -0.200675

        return (a + b * T + c * T ** 2 + D * T ** 3 + E / T ** 2) / 58.44

# NaCl + Water Specific Heat
    def cpm(T,c):
        return cpw(T) * (1-c) + cps(T) * c    

# Hot Side Convective Heat Coeff. [kJ/m2 s]
    def h_hs(T,mf,x,y):
            Mass_Flow = mf
            Vol_Flow = Mass_Flow / den(T)
            Area = x * y
            Perimeter = 2 * x + 2 * y
            d_h = (4 * Area) / (Perimeter)
            vis_f = k_vis(T)
            vel_f = Vol_Flow / Area
            Cp = cpw(T)
            Re = vel_f * d_h / vis_f
            Pr = Cp * d_vis(T) / k_f(T)

            Nuh = 0.023 * Re**0.8 * Pr**0.4

            return Nuh * k_f(T) / d_h

# Cold Side Convective Heat Coeff. [kJ/m2 s]
    def h_cs(T,mf,x,y):
            Mass_Flow = mf
            Vol_Flow = Mass_Flow / den(T)
            Area = x * y
            Perimeter = 2 * x + 2 * y
            d_h = (4 * Area) / (Perimeter)
            vis_f = k_vis(T)
            vel_f = Vol_Flow / Area
            Cp = cpw(T)
            Re = vel_f * d_h / vis_f
            Pr = Cp * d_vis(T) / k_f(T)

            Nuh = 0.023 * Re**0.8 * Pr**0.3

            return Nuh * k_f(T) / d_h

        
# Average Nacl diffusion coefficient [m2/s]
    def Dnacl(T):
        return 1e-9

# Mass Transfer Coeff. [kJ/m2 s]
    def kf(T,mf,x,y):
        Mass_Flow = mf
        Vol_Flow = Mass_Flow / den(T)
        Area = x * y
        Perimeter = 2 * x + 2 * y
        d_h = (4 * Area) / (Perimeter)
        vis_f = k_vis(T)
        vel_f = Vol_Flow / Area
        Cp = cpw(T)
        Re = vel_f * d_h / vis_f
        Sc =  vis_f / Dnacl(T)

        Sh = 0.023 * Re**0.8 * Sc**0.3

        return Sh * Dnacl(T) / d_h


# Membrane Mass Trasnfer Coeff. [kg/m2 Pa s] 
    def Cm(Tmf,Tmp,c_salt):
        k_B = 1.38E-23                                                  #[J/K]    Boltzmann constant
        T_m = (Tmf + Tmp) / 2 + 273.15                                  #[C]      Mean membrane Temperature
        P_m = (antoine(Tmf) * wa(c_salt) + antoine(Tmp))/ 2             #[Pa]     Mean membrane Pressure
        sigma = (2.641+3.711)/2*1e-10                                   #[m]      Collision diameter of the vapour water molecule
        mu = 0.000354                                                   #[Pa s]   vapour viscosicty
        Ptot = 101325                                                   #[Pa]     P Air
        Pair = Ptot - P_m                                               #[Pa]     P Total (Air + water Vapour)
        Dvap = ((T_m)**2.072) * 0.00001895/Ptot                         #[m2/s]   Diffusion coefficient of vapour

        Pi =np.pi                                                       #         Pi Number
        l = k_B * T_m / ((2**0.5) * Pi * Ptot * (sigma**2))             #[m]      Mean free path traveled by the transported molecule
        kn = l / dp                                                     #         Knudsen Number

        P_s = dp                                                        #[m]      pore size
        Tort = ((2 - Por)**2) / Por                                     #[-]      tortuosity
        Thick_m = em                                                    #[m]      thickness
        mi = 18.01528 / 1000                                            #[kg/mol] Molecular weight (permeating component[water])
        R = 8.314472                                                    #[m3 Pa/(mol K)] gas constant


        C_kn = (1 / 3) * (Por * P_s / (Tort * Thick_m)) * (8 * mi / (Pi * R * T_m))**0.5

        C_vis = (1 / 8 * mu) * (Por * P_s**2 / (Tort * Thick_m)) * (mi * P_m / (R * T_m))

        C_mol = (Por / (Tort * Thick_m)) * (mi / (R * T_m)) * (Ptot * Dvap / Pair)

        C_mem = (1 / C_kn + 1 / C_mol)**-1
    
        return C_mem   
    
# Membrane Water Permeate 
    def Permeate(THm,TCm,c_salt):
        return Cm(THm,TCm,c_salt)*Darea*(antoine(THm)*wa(c_salt*100)-antoine(TCm))


    def matrix(z):

        TH  = z[0:N]           #Hot feed temperature profile
        THm = z[N:2*N]         #Hot membrane side temperature profile 
        TCm = z[2*N:3*N]       #Cold membrane side temperature profile 
        TC  = z[3*N:4*N]       #Cold feed temperature profile 
        CM  = z[4*N:]          #Surface membrane concentration profile

        F = np.empty(5*N)      #Balance array
        

        for i in range(N): 
            
# Balance in top of the module
            if i==0:
                F[0] = FH0*cpm(TH0,c_salt)*(TH0-TH[0]) - h_hs(TH0,FH0,W,H)*Darea*(TH0-THm[0]) 
                F[1] = h_hs(TH0,FH0,W,H)*Darea*(TH0-THm[0]) - (Permeate(THm[0],TCm[0],c_salt)*DH + km/em*(THm[0]-TCm[0])*Darea)   
                F[2] = Permeate(THm[0],TCm[0],c_salt)*DH + km/em*(THm[0]-TCm[0])*Darea - h_cs(TC[1],FC0,W,H)*Darea*(TCm[0]-TC[1])
                F[3] = FC0*cpw(TC[0])*(TC[0]-TC[1]) - h_cs(TC[1],FC0,W,H)*Darea*(TCm[0]-TC[1])
                F[4] = CM[i]-c_salt*np.exp(Permeate(THm[0],TCm[0],CM[i])/(den(TH0)*kf(TH0,FH0,W,H)))
                
            elif i==N-1:
                F[5*N-5] = FH0*cpm(TH[i-1],c_salt)*(TH[i-1]-TH[i]) - h_hs(TH[i-1],FH0,W,H)*Darea*(TH[i-1]-THm[i])  
                F[5*N-4] = h_hs(TH[i-1],FH0,W,H)*Darea*(TH[i-1]-THm[i]) - (Permeate(THm[i],TCm[i],c_salt)*DH + km/em*(THm[i]-TCm[i])*Darea)    
                F[5*N-3] = Permeate(THm[i],TCm[i],c_salt)*DH + km/em*(THm[i]-TCm[i])*Darea - h_cs(TC0,FC0,W,H)*Darea*(TCm[i]-TC0)
                F[5*N-2] = FC0*cpw(TC[i])*(TC[i]-TC0) - h_cs(TC0,FC0,W,H)*Darea*(TCm[i]-TC0)
                F[5*N-1] = CM[i]-c_salt*np.exp(Permeate(THm[0],TCm[0],CM[i])/(den(TH[i-1])*kf(TH[i-1],FH0,W,H)))

# Balance in bottom of the module                
            else:
                F[5*i]   = FH0*cpm(TH[i-1],c_salt)*(TH[i-1]-TH[i]) - h_hs(TH[i-1],FH0,W,H)*Darea*(TH[i-1]-THm[i]) 
                F[5*i+1] = h_hs(TH[i-1],FH0,W,H)*Darea*(TH[i-1]-THm[i]) - (Permeate(THm[i],TCm[i],c_salt)*DH + km/em*(THm[i]-TCm[i])*Darea)   
                F[5*i+2] = Permeate(THm[i],TCm[i],c_salt)*DH + km/em*(THm[i]-TCm[i])*Darea - h_cs(TC[i+1],FC0,W,H)*Darea*(TCm[i]-TC[i+1])
                F[5*i+3] = FC0*cpw(TC[i])*(TC[i]-TC[i+1]) - h_cs(TC[i+1],FC0,W,H)*Darea*(TCm[i]-TC[i+1])
                F[5*i+4] = CM[i]-c_salt*np.exp(Permeate(THm[0],TCm[0],CM[i])/(den(TH[i-1])*kf(TH[i-1],FH0,W,H)))
                
        return F
# Initial guess array
    zguess = np.ones(N*5)

# Roots return of the array
    z = fsolve(matrix,zguess)

    TH  = z[0:N]
    THm = z[N:2*N]
    TCm = z[2*N:3*N]
    TC  = z[3*N:4*N]
    CM  = z[4*N:]
    
    
# Permeate results
    P=np.ones(N)
    
    for i in range(N):
        P[i]=Permeate(THm[i],TCm[i],CM[i])
        
# Permeate results
    return TH,THm,TCm,TC,P,CM



#                                                       DCMD Sample

# DCMD(FH0[kg/s],FC0[kg/s],C0_salt[w/w],TH0[ºC],TC0[ºC],L[m],W[m],H[m],NºSteps)

FH0=1/60                         # Initial mass flow of hot flow    [kg/s]
FC0=1/60                         # Initial mass flow of cold flow   [kg/s]
TH0=81                           # Initial Temperature of hot flow  [°C]
TC0=21                           # Initial Temperature of cold flow [°C]
c0_salt=0                        # Initial Salt mass fraction concentration [w/w]
L,W,H=[.114,.025,0.0015]         # Membrane Length,Width,Height
# L,W,H=[.06,.015,0.001]
N=20                             # Number of divisions
sample1=DCMD(FH0,FC0,c0_salt,TH0,TC0,L,W,H,N)

print('Permeate:',"%.1f" % (np.sum(sample1[4])*3.6e6),"g/h")
print('Flux:',"%.1f" % (np.sum(sample1[4])*3.6e3/(W*L)),"kg/h/m2")

# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("Ready at", dt_string)